Okay, this is a little tricky, let's try to figure this out.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr

In [2]:
# path to dataframes
path_to_dfs = "forecasted_counts/block_0103/"

In [3]:
# os.listdir(path_to_dfs)

In [4]:
# choose only the average forecasts dataframes
averaged_forecasts_csvs = []
for file in os.listdir(path_to_dfs):
    if file[:8] =='averaged':
        averaged_forecasts_csvs.append(file)

In [5]:
len(averaged_forecasts_csvs)

42

In [6]:
averaged_forecasts_csvs.sort() # - note these are not sorted, we need these sorted to match the correct counts to the correct subwindows

In [7]:
averaged_forecasts_csvs[0]

'averaged_forecasts_sub_0.csv'

In [8]:
# order these csv files, let's write a for loop
ordered_csv_files = ['averaged_forecasts_sub_' + str(i) + '.csv' for i in np.arange(42)]

In [9]:
# ordered_csv_files

In [10]:
all_dfs = []
for file in ordered_csv_files:
    read_df = pd.read_csv(path_to_dfs + '/' + file)
    all_dfs.append(read_df)

In [11]:
# make a single dataframe out of the many
combined_df = pd.concat(all_dfs, axis = 1)

In [12]:
combined_df.head()

,True_value,Forecasted_value,True_value,Forecasted_value,True_value,Forecasted_value,True_value,Forecasted_value,True_value,Forecasted_value,...,True_value,Forecasted_value,True_value,Forecasted_value,True_value,Forecasted_value,True_value,Forecasted_value,True_value,Forecasted_value
0,3.000000,3.518405,5.680015,4.711437,4.008115,4.301618,2.211300,3.509243,6.203186,5.859481,...,0.000531,0.079276,0.000674,0.027422,0.000142,0.070028,0.000000,0.072067,0.0,-0.000362
1,4.002516,3.732567,6.450665,4.825813,3.788700,4.349662,4.397723,2.930528,7.895105,5.337866,...,0.000674,0.061360,0.000000,-0.082306,0.000000,-0.023784,0.000000,0.048017,0.0,-0.000514
2,4.000155,3.428718,5.915424,5.178430,4.004599,4.501003,3.997293,3.555890,6.674109,5.828358,...,0.000000,0.085696,0.000000,0.027619,0.000005,0.110478,0.000005,0.065582,0.0,-0.000339
3,3.000000,4.329351,6.666009,5.262213,4.000000,4.504326,1.000000,2.507873,4.000031,6.039442,...,0.000000,0.058320,0.000000,-0.008478,0.000674,0.001183,0.000674,0.043805,0.0,0.000566
4,3.000000,3.944138,5.671839,5.235770,4.022552,4.438399,2.008101,2.936499,4.633390,5.194905,...,0.000000,0.066148,0.000000,-0.005671,0.004599,-0.036351,0.004599,0.028059,0.0,-0.000025


In [13]:
combined_df.shape

(7, 84)

In [14]:
# get the true values
True_values_df = combined_df[['True_value']]

In [15]:
True_values_df

,True_value,True_value,True_value,True_value,True_value,True_value,True_value,True_value,True_value,True_value,...,True_value,True_value,True_value,True_value,True_value,True_value,True_value,True_value,True_value,True_value
0,3.000000,5.680015,4.008115,2.211300,6.203186,6.000008,2.000008,2.991899,3.841271,3.001654,...,3.999930,1.999930,0.000000,0.000000,0.000000,0.000531,0.000674,0.000142,0.000000,0.0
1,4.002516,6.450665,3.788700,4.397723,7.895105,5.840550,1.922893,5.020036,6.011305,3.000000,...,4.321506,2.000000,0.000000,0.000000,0.000674,0.000674,0.000000,0.000000,0.000000,0.0
2,4.000155,5.915424,4.004599,3.997293,6.674109,4.788695,0.999997,3.999930,5.547751,3.022552,...,5.999987,4.000000,0.000013,0.000000,0.000000,0.000000,0.000000,0.000005,0.000005,0.0
3,3.000000,6.666009,4.000000,1.000000,4.000031,3.500031,0.500000,3.996810,5.876725,3.000013,...,4.080343,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000674,0.000674,0.0
4,3.000000,5.671839,4.022552,2.008101,4.633390,3.633390,0.985549,2.000000,3.211300,3.233852,...,4.991866,3.000000,0.008102,0.000000,0.000000,0.000000,0.000000,0.004599,0.004599,0.0
5,4.000000,8.271483,6.054452,2.054460,5.051951,5.000003,1.948060,2.000000,3.054791,4.076859,...,5.995371,4.999969,1.000000,0.000000,0.000000,0.000000,0.002516,0.010618,0.008102,0.0
6,1.000000,4.974901,4.500000,2.681041,4.155932,2.002517,0.002526,1.991898,4.967989,2.999985,...,3.000031,1.977448,0.000000,0.004599,0.004587,0.000000,0.000509,0.077012,0.076503,0.0


In [16]:
# since the subwindows are now overlapping, we cannot directly add the counts - These needs to be now normalized - might have to think this through

In [17]:
# sum the true values - since the sub images are not overlapping, we can get the sum across columns in the above dataframe
sum_true_values = True_values_df.sum(axis = 1)

In [18]:
sum_true_values # note that these values do not match the actual counts we have - these needs to be properly normalized
# We need to write out the code for proper normalization

0    133.547655
1    126.067732
2    139.849757
3    107.537386
4    104.110652
5    131.280152
6     97.747947
dtype: float64

In [19]:
# separate the forecasted values

In [20]:
# get the true values
Forecasted_values_df = combined_df[['Forecasted_value']]

In [21]:
Forecasted_values_df

,Forecasted_value,Forecasted_value,Forecasted_value,Forecasted_value,Forecasted_value,Forecasted_value,Forecasted_value,Forecasted_value,Forecasted_value,Forecasted_value,...,Forecasted_value,Forecasted_value,Forecasted_value,Forecasted_value,Forecasted_value,Forecasted_value,Forecasted_value,Forecasted_value,Forecasted_value,Forecasted_value
0,3.518405,4.711437,4.301618,3.509243,5.859481,4.431803,1.223143,3.454632,5.072189,3.500179,...,4.111874,3.117257,-0.143978,0.090642,0.116495,0.079276,0.027422,0.070028,0.072067,-0.000362
1,3.732567,4.825813,4.349662,2.930528,5.337866,4.609105,1.159936,3.937839,5.449829,3.497967,...,4.127012,3.204715,-1.539469,0.070775,0.104766,0.061360,-0.082306,-0.023784,0.048017,-0.000514
2,3.428718,5.178430,4.501003,3.555890,5.828358,4.907714,1.485616,3.768842,5.148653,3.524544,...,4.546917,3.812811,0.697895,0.060483,0.098122,0.085696,0.027619,0.110478,0.065582,-0.000339
3,4.329351,5.262213,4.504326,2.507873,6.039442,5.135900,0.994300,6.190042,9.061913,3.737752,...,4.310143,2.468361,-2.751352,0.083884,0.083081,0.058320,-0.008478,0.001183,0.043805,0.000566
4,3.944138,5.235770,4.438399,2.936499,5.194905,4.869748,1.130439,5.732378,6.953180,3.626733,...,4.280945,2.720168,-0.816729,0.015056,0.042080,0.066148,-0.005671,-0.036351,0.028059,-0.000025
5,3.820706,4.961481,4.429846,2.712862,5.501562,4.671822,1.040842,4.411165,6.224572,3.858034,...,4.149870,2.854491,-0.997376,0.039747,0.101527,0.044316,-0.211945,-0.007291,0.050153,0.000234
6,3.895215,5.195496,4.578771,2.856078,5.805936,5.756642,1.113414,4.276563,6.367151,3.767047,...,4.234437,2.771484,-2.655980,0.064651,0.043874,0.060493,0.040351,0.054170,0.012705,0.000344


In [22]:
sum_forecasted_values = Forecasted_values_df.sum(axis = 1)

In [23]:
sum_forecasted_values

0    120.083619
1    116.538534
2    134.260614
3    120.410041
4    122.147701
5    120.904704
6    121.613581
dtype: float64

In [24]:
# below is the code from previous work. we need to normalize both true and forecasted counts

In [25]:
Im_6_true_val_list = list(True_values_df.iloc[6,:])

In [26]:
Im_0_pred_val_list = list(Forecasted_values_df.iloc[0,:])

In [27]:
# we also need the correct image height and width
image_path = '/common/statsgeneral/gayara/Spring_2024/S_lab_TasselNet/Block_1_TN/Block_1_images_and_xml/Block0101_2020_09_16.jpeg'

In [28]:
read_image = plt.imread(image_path)

In [29]:
read_image.shape

(768, 1024, 3)

In [30]:
img_height = read_image.shape[0]
img_width = read_image.shape[1]

In [31]:
# define a function to get the post-hoc prediction
def normalized_counts_on_test_data(values_list, img_height, img_width, stride = 150, kernel_size = 300):
    # create the empty count and density maps
    # density map
    Density_map = np.zeros((img_height, img_width))

    # counts map
    counts_map = np.zeros((img_height, img_width))
    
    # now, for every window, we will keep adding the values together and also add the counts
    counter = 0
#     need a counter to move into each predicted value in the pred values list
    for ii in range(0, img_height, stride):
        for jj in range(0, img_width, stride):
#         operations for density map
#             get the window of interest
            new_window = Density_map[ii:ii + kernel_size,jj:jj+kernel_size]
#     fill each with the value c_k
            counts_window = np.full((new_window.shape[0], new_window.shape[1]), values_list[counter])
#     get the shapes of this new window
            cw_height = counts_window.shape[0]
            cw_width = counts_window.shape[1]
#         Do c_k/r_2
            counts_window_new = counts_window/(cw_height*cw_width)
#     This is the value in the window now
            value_window = counts_window_new
#     place the values in the corrsponding area of the density map
            Density_map[ii:ii + kernel_size,jj:jj+kernel_size] = new_window + value_window

#         Let's now focus on capturing the counts of the windows
            new_window_c = counts_map[ii:ii + kernel_size,jj:jj+kernel_size]
#     get the counts area
            count = np.ones((new_window_c.shape[0], new_window_c.shape[1]))
#     keep adding the counts to reflect the addition of densities
            counts_map[ii:ii + kernel_size,jj:jj+kernel_size] = new_window_c + count
#     increase the counter
            counter = counter + 1
            
#         get the normalized count
    normalized_counts = np.divide(Density_map, counts_map)

#     entire count on the test set
    pred_on_test = np.sum(normalized_counts)
    
#     return the predicted value
    return(pred_on_test, normalized_counts)

In [32]:
%%time
# get the true counts in a loop based on normalization stratergy
all_true_values_block_0103 = []
for i in range(combined_df.shape[0]):
    true_value_list = list(True_values_df.iloc[i,:])
    norm_values = normalized_counts_on_test_data(true_value_list, img_height, img_width, stride = 150, kernel_size = 300)
    all_true_values_block_0103.append(norm_values[0])

CPU times: user 133 ms, sys: 15.9 ms, total: 149 ms
Wall time: 149 ms


In [33]:
all_true_values_block_0103 

[38.711229894695975,
 38.432178320042986,
 41.384982904395834,
 31.53002132885738,
 30.70951560861435,
 38.49258916592229,
 28.163088439177926]

In [34]:
# These match with the true values from earlier (more or less)
# 0    40.000660
# 1    39.000002
# 2    41.000000
# 3    31.000000
# 4    32.000000
# 5    40.002085
# 6    27.000176

In [35]:
%%time
# get the forecasted counts in a loop
all_pred_values_block_0103 = []
for i in range(combined_df.shape[0]):
    pred_value_list = list(Forecasted_values_df.iloc[i,:])
    norm_values = normalized_counts_on_test_data(pred_value_list, img_height, img_width, stride = 150, kernel_size = 300)
    # save the normalized map for later 
    save_name = 'normalized_maps/block_0103/' + 'normalized_count_map_image_' + str(i) + '.npy'
    np.save(save_name, norm_values[1])
    all_pred_values_block_0103.append(norm_values[0])

CPU times: user 185 ms, sys: 11.4 ms, total: 197 ms
Wall time: 425 ms


In [36]:
all_pred_values_block_0103

[35.775436225612474,
 34.937822217782504,
 39.533600742524996,
 36.51786241223245,
 36.71194243906225,
 36.12734765407499,
 36.54486812147502]

In [37]:
# # well we need to verify that the above true values are indeed correct
# from preprocess script we have the following counts
# [43, 49, 53, 59, 45, 42, 34, 39, 37, 43, 41, 39, 43, 40, 39, 41, 31, 32, 40, 29] # the last seven numbers match with what we have.

In [38]:
# blockwise mean squared error
rmse = np.sqrt(mean_squared_error(all_true_values_block_0103, all_pred_values_block_0103))
rmse

4.796029749787816

In [39]:
# blockwise mean absolute error
mae = mean_absolute_error(all_true_values_block_0103, all_pred_values_block_0103)
mae

4.28840300588317

In [40]:
corr = pearsonr(all_true_values_block_0103, all_pred_values_block_0103)
corr

PearsonRResult(statistic=0.19513058868043867, pvalue=0.6750064343261976)

In [41]:
# get the rest of the metrics

In [42]:
# Okay, now what about the rest?
# we are on the path
path_to_dfs = "forecasted_counts/block_0103/"

In [43]:
all_contents = os.listdir(path_to_dfs)

In [44]:
# we need the npy files for all forecasts
all_forecast_files = [file for file in all_contents if file[:3] == 'all']

In [45]:
# all_forecast_files
all_forecast_files[0]

'all_forecasts_sub_11.npy'

In [46]:
# need these in sorted order
final_all_forecast_files = ['all_forecasts_sub_' + str(i) + '.npy' for i in range(42)]

In [47]:
# final_all_forecast_files

In [48]:
# Okay, so these needs to be normalized too, when finding the coverages and widths - seems like a tedious task, will get to this in the next round of 2 hour work

In [49]:
# load these npy files
loaded_npy_files = []
for file in final_all_forecast_files:
    joined_path = os.path.join(path_to_dfs, file)
    load_file = np.load(joined_path)
    loaded_npy_files.append(load_file)

In [50]:
loaded_npy_files[0].shape

(1000, 4, 7)

In [51]:
len(loaded_npy_files)

42

In [52]:
# we cannot add these up now like we used to do earlier, need to think how to tackle the arrays

# reshape the (1000, 4) to 4000 to aid with the normalization?
reshaped_npy_files = []
for file in loaded_npy_files:
    reshaped_file = file.reshape(4000,7)
    reshaped_npy_files.append(reshaped_file)

In [53]:
reshaped_npy_files[0].shape

(4000, 7)

In [54]:
## Check the shaped of all the files
# for i in reshaped_npy_files:
#     print(i.shape)

# length of this list
len(reshaped_npy_files)

42

In [55]:
# note that over here, for each of the test image, we should have 4000 final normalized predited values like earlier, and we can find the coverages and the widths

In [56]:
# join all these together to get the predictions
stacked_reshaped_files = np.stack(reshaped_npy_files)

In [57]:
# see the shape
stacked_reshaped_files.shape

(42, 4000, 7)

In [58]:
# verify for random arrays the stacking is correctly done
np.mean(reshaped_npy_files[18] == stacked_reshaped_files[18])

1.0

In [59]:
# okay, now the image is in the final dimension of the shape - we have 7 test images

# check the shape of the first image
stacked_reshaped_files[:,:,0].shape

(42, 4000)

In [60]:
# check the shape of the final image
stacked_reshaped_files[:,:,6].shape

(42, 4000)

In [61]:
# define n chains as the number of predicted values
n_chains = reshaped_npy_files[0].shape[0]
n_chains

4000

In [62]:
# do this for a single image before doing a function to use for any image

# do this for the 0th test image
chosen_preds = stacked_reshaped_files[:,:,0]

In [63]:
chosen_preds.shape

(42, 4000)

In [64]:
%%time
all_preds_im_0 = []
for chain in np.arange(chosen_preds.shape[-1]):
    pred_val = normalized_counts_on_test_data(chosen_preds[:,chain], img_height, img_width, stride = 150, kernel_size = 300)
    all_preds_im_0.append(pred_val[0])

CPU times: user 1min 11s, sys: 6.26 ms, total: 1min 11s
Wall time: 1min 12s


In [65]:
len(all_preds_im_0)

4000

In [66]:
np.mean(all_preds_im_0)

35.77543742497176

In [67]:
%%time
# okay, now need to do this for all images

all_image_preds = []
for image in np.arange(stacked_reshaped_files.shape[-1]):
    img_specific_preds = []
    chosen_pred_vals = stacked_reshaped_files[:,:, image]
    for chain in np.arange(chosen_pred_vals.shape[-1]):
        pred_val = normalized_counts_on_test_data(chosen_pred_vals[:,chain], img_height, img_width, stride = 150, kernel_size = 300)
        img_specific_preds.append(pred_val[0])
    all_image_preds.append(img_specific_preds)

CPU times: user 8min 10s, sys: 33.5 ms, total: 8min 10s
Wall time: 8min 13s


In [68]:
# okay the final output should be of shape (4000, 7)

In [69]:
all_preds_array = np.array(all_image_preds)
all_preds_array = np.swapaxes(all_preds_array, 0,1)

In [70]:
all_preds_array.shape

(4000, 7)

In [71]:
li_train = np.percentile(all_preds_array, axis = 0, q = (2.5, 97.5))[0,:].reshape(-1,1)    
ui_train = np.percentile(all_preds_array, axis = 0, q = (2.5, 97.5))[1,:].reshape(-1,1)

In [72]:
li_train

array([[28.05195842],
       [27.69461452],
       [31.47349856],
       [25.65850897],
       [26.49711843],
       [26.4090642 ],
       [25.95994394]])

In [73]:
ui_train

array([[41.81290128],
       [40.98144151],
       [45.68265617],
       [47.51094283],
       [46.75503375],
       [45.35010598],
       [46.83524425]])

In [74]:
width_train = ui_train - li_train
avg_width_train = width_train.mean(0)[0]

In [75]:
avg_width_train

17.597659821426884

In [76]:
y_traina = all_true_values_block_0103

In [77]:
ind_train = (y_traina >= li_train) & (y_traina <= ui_train)
coverage_train= ind_train.mean(0)[0]
coverage_train

1.0

In [78]:
# See if the means across the 4000 values for the images match with the normalized counts we have

In [79]:
all_preds_array.mean(axis = 0)

array([35.77543742, 34.93782544, 39.53359887, 36.51786978, 36.71195072,
       36.12734718, 36.54487384])

In [80]:
# # values from earlier for comparison
# [35.775436225612474,
#  34.937822217782504,
#  39.533600742524996,
#  36.51786241223245,
#  36.71194243906225,
#  36.12734765407499,
#  36.54486812147502]

In [81]:
# They do match, so I believe the normalization we have written works good